In [1]:
import hail as hl
hl.init(log='mouse_simulation.log')

Running on Apache Spark version 2.4.1
SparkUI available at http://192.168.0.79:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.19-c6ec8b76eb26
LOGGING: writing to mouse_simulation.log


In [2]:
high = hl.read_table('./all_high_array.ht')
high_or_med = hl.read_table('./all_high_med_array.ht')

In [3]:
# Schema of table
high.describe()

----------------------------------------
Global fields:
    'ids': array<str> 
----------------------------------------
Row fields:
    'mice': array<int32> 
----------------------------------------
Key: []
----------------------------------------


In [4]:
# All mice available in tables
hl.eval(high.ids)

['129P2_OlaHsd',
 '129S1_SvImJ',
 '129S5SvEvBrd',
 'AKR_J',
 'A_J',
 'BALB_cJ',
 'BTBR_T+_Itpr3tf_J',
 'BUB_BnJ',
 'C3H_HeH',
 'C3H_HeJ',
 'C57BL_10J',
 'C57BL_6NJ',
 'C57BR_cdJ',
 'C57L_J',
 'C58_J',
 'CAST_EiJ',
 'CBA_J',
 'DBA_1J',
 'DBA_2J',
 'FVB_NJ',
 'I_LnJ',
 'KK_HiJ',
 'LEWES_EiJ',
 'LP_J',
 'MOLF_EiJ',
 'NOD_ShiLtJ',
 'NZB_B1NJ',
 'NZO_HlLtJ',
 'NZW_LacJ',
 'PWK_PhJ',
 'RF_J',
 'SEA_GnJ',
 'SPRET_EiJ',
 'ST_bJ',
 'WSB_EiJ',
 'ZALENDE_EiJ']

In [5]:
# not all mice are commercially available. The available ones are below:
available_mice = [
    'BALB_cJ',
    'CBA_J',
    'CAST_EiJ',
    'NOD_ShiLtJ',
    'FVB_NJ',
    'C57BL_6NJ',
    'C57L_J',
    'DBA_2J',
    'AKR_J',
    'LP_J',
    'NZB_B1NJ',
    'MOLF_EiJ',
    'PWK_PhJ',
    '129S1_SvImJ',
    'A_J',
    'BTBR_T+_Itpr3tf_J',
    'NZO_HlLtJ',
    'NZW_LacJ',
    'C3H_HeJ',
    'C57BL_10J',
    'C57BR_cdJ',
    'DBA_1J',
    'LEWES_EiJ',
    'SPRET_EiJ',
    'WSB_EiJ',
]

In [6]:
from numpy.random import beta, shuffle, uniform
import numpy as np
from pprint import pprint


def print_header():
    print(f"""{"N":>4}{"MEAN":>8}{"SD":>8}{"MED":>8}{".05":>8}{".95":>8}""")
    print(f"""{"-":>4}{"----":>8}{"--":>8}{"---":>8}{"---":>8}{"---":>8}""")


def ff(x):  # format float
    return f'{x:.1f}'


def simulate(ht, n_mice, replicates):
    assert n_mice >= 2
    meta = []
    result = []
    to_do = replicates

    # mapping from name to position in the table's array
    table_mapping = hl.eval(
        hl.dict(hl.zip_with_index(ht.ids).map(lambda t: (t[1], t[0]))))

    def index(l):
        new_l = []
        for name in l:
            idx = table_mapping[name]
            assert idx < 36, idx
            new_l.append(idx)
        return new_l

    pos_neg_array = []  # array of (positive, negative) mouse indices

    for i in range(replicates):
        pos = []  # c57 is the reference
        neg = ['BALB_cJ']
        candidates = [
            m for m in available_mice if m not in pos and m not in neg]
        shuffle(candidates)

        # draw phenotype from a beta distribution, using prior of 1 pos, 1 neg
        p = beta(2, 2)

        for i in range(n_mice - 2):
            # sample individual mouse phenotypes according to `p`
            draw = uniform(0, 1)
            if draw < p:
                pos.append(candidates[i])
            else:
                neg.append(candidates[i])
        pos_neg_array.append((index(pos), index(neg)))
        meta.append((len(pos), len(neg), pos, neg, p))

    ht = ht.annotate_globals(pos_neg_array=hl.literal(pos_neg_array,
                                                      dtype='array<tuple(array<int>, array<int>)>'))
    result = ht.aggregate(
        hl.agg.array_agg(array=hl.range(0, replicates),
                         f=lambda i: hl.agg.count_where(ht.pos_neg_array[i][0].all(lambda x: ht.mice[x] == 0) &
                                                        ht.pos_neg_array[i][1].all(lambda x: ht.mice[x] == 2))))
    pct = np.nanpercentile(result, [5, 50, 95])
    print(f'{n_mice:>4}'
          f'{ff(np.mean(result)):>8}'
          f'{ff(np.std(result)):>8}'
          f'{ff(pct[1]):>8}'
          f'{ff(pct[0]):>8}'
          f'{ff(pct[2]):>8}')

In [7]:
# run simulation for high and medium consequence variants
print_header()
for i in range(3, 15):
    simulate(high_or_med, i, 1000)

   N    MEAN      SD     MED     .05     .95
   -    ----      --     ---     ---     ---
   3  8762.6  3383.6  8572.0  3746.0 13706.0
   4  4301.5  2594.7  3906.5   312.7  8872.6
   5  2202.2  1915.6  1686.0    36.0  6180.1
   6  1234.4  1269.0   800.0     4.0  4028.5
   7   733.3   939.6   378.5     1.0  2671.1
   8   363.8   576.2   134.5     0.0  1602.3
   9   236.5   470.7    63.0     0.0  1150.1
  10   139.0   303.8    27.0     0.0   803.0
  11    78.8   195.0     8.0     0.0   517.6
  12    56.2   157.8     2.0     0.0   381.6
  13    36.8   112.1     1.0     0.0   206.9
  14    26.9    92.8     0.0     0.0   139.4


In [8]:
# run simulation for high consequence variants
print_header()
for i in range(3, 15):
    simulate(high, i, 1000)

   N    MEAN      SD     MED     .05     .95
   -    ----      --     ---     ---     ---
   3   194.5    86.4   184.0    70.0   336.0
   4    88.6    60.7    73.0    15.0   212.0
   5    45.7    42.2    33.0     2.0   141.0
   6    23.8    26.2    15.0     0.0    75.0
   7    13.8    18.7     7.5     0.0    54.0
   8     7.8    14.1     2.0     0.0    31.0
   9     4.6     9.7     1.0     0.0    19.0
  10     2.9     7.0     0.0     0.0    14.0
  11     1.8     4.5     0.0     0.0    10.0
  12     1.0     3.3     0.0     0.0     7.0
  13     0.9     2.6     0.0     0.0     7.0
  14     0.6     2.1     0.0     0.0     5.0
